In [83]:
# import requirements
import os
import json
import sagemaker
import numpy as np
import pandas as pd
from sagemaker import get_execution_role

# get current SageMaker session's execution role and default bucket name
sagemaker_session = sagemaker.Session()

role = get_execution_role()
print("execution role ARN:", role)

# bucket_name_default = sagemaker_session.default_bucket()
# print(bucket_name_default)
# bucket_name = 'ml-demo-common'
%store -r bucket_name
print(bucket_name)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
execution role ARN: arn:aws:iam::629515838455:role/service-role/AmazonSageMaker-ExecutionRole-20231201T204975
sagemaker-gacheon-ml2-team1


In [84]:
training_data_uri = os.path.join(f's3://{bucket_name}', 'data')
print(training_data_uri)

s3://sagemaker-gacheon-ml2-team1/data


In [85]:
!pygmentize 'knn_model.py'

import argparse
import joblib
import os

import numpy as np
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import glob

# inference functions ---------------
def model_fn(model_dir):
    clf = joblib.load(os.path.join(model_dir, "model.joblib"))
    return clf


if __name__ == "__main__":
    print("extracting arguments")
    parser = argparse.ArgumentParser()

    # 현재 작업 디렉토리 내의 파일 목록 얻기
    file_list = os.listdir("/opt/ml/input/data/training")
    print(file_list)

    # 결과 출력
    print("현재 작업 디렉토리의 파일 및 디렉토리 목록:")
    for file in file_list:
        print(file)

    # hyperparameters sent by the client are passed as command-line arguments to the script.
    # to simplify the demo we don't use all sklearn RandomForest hyperparameters
    parser.add_argument("--n_neighbors", type=int, default=5)
    
    # Data, model, and output directories
    parser.add_argument("--model-dir", type=str, default=os.environ.get("SM_MO

In [88]:
# specify training instance type and model hyperparameters
# note that for the demo purpose, the number of epoch is set to 1

num_of_instance = 1                 # number of instance to use for training
instance_type = 'ml.m5.large'     # type of instance to use for training

training_script = 'knn_model.py'

training_parameters = {
    'n_neighbors':3,
    'user_id':97
}

# training framework specs
tensorflow_version = '2.1.0'
python_version = 'py3'
distributed_training_spec = {'parameter_server': {'enabled': True}}
FRAMEWORK_VERSION = "0.23-1"

In [89]:
from sagemaker.sklearn.estimator import SKLearn
sklearn_estimator = SKLearn(
    entry_point=training_script,
    role=role,
    instance_count=num_of_instance,
    instance_type=instance_type,
    framework_version=FRAMEWORK_VERSION,
    base_job_name="rf-scikit",
    hyperparameters=training_parameters,
)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [ ]:
# knn = sagemaker.estimator.Estimator(
#     get_image_uri(boto3.Session().region_name, "knn"),
#     role=role,
#     instance_count=num_of_instance,
#     instance_type=instance_type,
#     output_path=output_path,
#     sagemaker_session=sagemaker.Session(),
#     hyperparameters=training_parameters,
# )

In [90]:
sklearn_estimator.fit(training_data_uri) 
# knn.fit(fit_input)

INFO:sagemaker:Creating training-job with name: rf-scikit-2023-12-04-22-48-13-315


Using provided s3_resource
2023-12-04 22:48:13 Starting - Starting the training job...
2023-12-04 22:48:28 Starting - Preparing the instances for training......
2023-12-04 22:49:25 Downloading - Downloading input data...
2023-12-04 22:50:00 Training - Downloading the training image...
2023-12-04 22:50:41 Uploading - Uploading generated training model.2023-12-04 22:50:36,382 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
2023-12-04 22:50:36,386 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2023-12-04 22:50:36,431 sagemaker_sklearn_container.training INFO     Invoking user training script.
2023-12-04 22:50:36,589 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2023-12-04 22:50:36,601 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2023-12-04 22:50:36,613 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2023-12-

In [91]:
import boto3
sklearn_estimator.latest_training_job.wait(logs="None")
sm_boto3 = boto3.client("sagemaker")
artifact = sm_boto3.describe_training_job(
    TrainingJobName=sklearn_estimator.latest_training_job.name
)["ModelArtifacts"]["S3ModelArtifacts"]

print("Model artifact persisted at " + artifact)


2023-12-04 22:50:57 Starting - Preparing the instances for training
2023-12-04 22:50:57 Downloading - Downloading input data
2023-12-04 22:50:57 Training - Training image download completed. Training in progress.
2023-12-04 22:50:57 Uploading - Uploading generated training model
2023-12-04 22:50:57 Completed - Training job completed
Model artifact persisted at s3://sagemaker-ap-northeast-2-629515838455/rf-scikit-2023-12-04-22-48-13-315/output/model.tar.gz


In [92]:
from sagemaker.sklearn.model import SKLearnModel

model = SKLearnModel(
    model_data=artifact,
    role=role,
    entry_point=training_script,
    framework_version=FRAMEWORK_VERSION,
)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [93]:
endpoint_name = 'perfume-recomand-ml2-last1'
model_name = 'perfume-recomand-model-ml2-last1'

predictor = model.deploy(instance_type=instance_type,
                         endpoint_name=endpoint_name,
                         model_name=model_name,
                         initial_instance_count=num_of_instance)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


INFO:sagemaker:Creating model with name: sagemaker-scikit-learn-2023-12-04-22-51-31-925
INFO:sagemaker:Creating endpoint-config with name perfume-recomand-ml2-last1
INFO:sagemaker:Creating endpoint with name perfume-recomand-ml2-last1


----!

In [94]:
predictor.predict([[3, 1, 31, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

array([136])